# Import Libraries

In [2]:
from pinecone.grpc import PineconeGRPC as PineconeGRPC_Client
from langchain_pinecone import Pinecone as LangChainPinecone


from pinecone import ServerlessSpec

from langchain_openai import OpenAI, OpenAIEmbeddings


from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()
from langchain.text_splitter import RecursiveCharacterTextSplitter


d:\DigiMark\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\.venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Loading The LLM (Language Model)


In [3]:
llm = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Setting Embeddings

In [4]:
embeddings=OpenAIEmbeddings(api_key='sk-proj-S1Geig55Gagb5NcCxPDjT3BlbkFJznF1JG9n5VEEMQF2sQJ6')

# Creating Pinecone Index

In [5]:
pc = PineconeGRPC_Client(api_key=os.environ.get('PINECONE_API_KEY'))

In [6]:
try:
  pc.create_index(
    name="uslaw",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
    )
  )
except Exception as err:
  print(err.body)


{"error":{"code":"FORBIDDEN","message":"Request failed. You've reached the max serverless indexes allowed in project Default (5). To add more serverless indexes, upgrade your plan."},"status":403}


In [7]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'testid-9g87gt2.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'testid',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 4096,
              'host': 'zain-9g87gt2.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'zain',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 4096,
              'host': 'immigration-9g87gt2.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'immigration',
              'spec': {'serverless': {'cloud': 'aws', 'regi

# Loading Documents

In [13]:
file_path = r'D:\DigiMark\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\monkey_story.txt'

In [14]:
loader = TextLoader(file_path) 

In [15]:
documents = loader.load()

In [16]:
documents[0].metadata

{'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\monkey_story.txt'}

# Loading Pinecone Vector Store

In [17]:
file_name = os.path.basename(file_path)
file_name

'monkey_story.txt'

In [18]:
pcv= LangChainPinecone(index_name='uslaw',embedding=embeddings,pinecone_api_key=os.environ.get('PINECONE_API_KEY'),namespace=file_name)

C:\Users\Predator\AppData\Local\Temp\ipykernel_26904\1410444277.py:1: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 0.3.0. Use :class:`~PineconeVectorStore` instead.
  pcv= LangChainPinecone(index_name='uslaw',embedding=embeddings,pinecone_api_key=os.environ.get('PINECONE_API_KEY'),namespace=file_name)


In [19]:
help(pcv)

Help on Pinecone in module langchain_pinecone.vectorstores object:

class Pinecone(PineconeVectorStore)
 |  Pinecone(index: 'Optional[Any]' = None, embedding: 'Optional[Embeddings]' = None, text_key: 'Optional[str]' = 'text', namespace: 'Optional[str]' = None, distance_strategy: 'Optional[DistanceStrategy]' = <DistanceStrategy.COSINE: 'COSINE'>, *, pinecone_api_key: 'Optional[str]' = None, index_name: 'Optional[str]' = None)
 |  
 |  .. deprecated:: 0.0.3 Use :class:`~PineconeVectorStore` instead.
 |  
 |  Deprecated. Use PineconeVectorStore instead.
 |  
 |  Method resolution order:
 |      Pinecone
 |      PineconeVectorStore
 |      langchain_core.vectorstores.base.VectorStore
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, index: 'Optional[Any]' = None, embedding: 'Optional[Embeddings]' = None, text_key: 'Optional[str]' = 'text', namespace: 'Optional[str]' = None, distance_strategy: 'Optional[DistanceStrategy]' = <DistanceStrategy.COSI

In [33]:
file_type = os.path.splitext(file_path)[1].replace('.', '')  # Get the file extension without the dot

for doc in documents:
    doc.metadata["file_type"] = file_type  # Add file type to metadata

In [34]:
documents

[Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\monkey_story.txt', 'file_type': 'txt'}, page_content="In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's gesture of friendship, Mango hugged Coco tightly, cherishing the bond they shared. From that day on, Coco and Mango ventured through the forest together, their friendship growing stronger with each passing adventure. As they watched the sun dip below the horizon, casting a golden glow over the treetops, they knew t

# Splitting Text into Chunks

In [35]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_documents(documents)

In [36]:
chunks[0]

Document(metadata={'source': 'D:\\DigiMark\\Developing-a-Legal-Document-Management-and-Chatbot-System-for-U.S.-Law\\monkey_story.txt', 'file_type': 'txt'}, page_content="In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's")

# Adding Documents to Pinecone

In [37]:
pcv.add_documents(chunks)

['a8388daa-51f9-49f4-91ba-3a90819d83a1',
 'f8392166-e00e-4dc5-a7c8-f5b5e8b1a570',
 'f58a60d6-e0e3-41e2-b38b-34821f0b129a']

# Creating a Retriever


In [18]:
retriever = pcv.as_retriever(search_type="similarity", search_kwargs={"k": 7})

# Chat Prompt

In [19]:

retrieval_qa_chat_prompt = PromptTemplate(
input_variables=["context", "input"],
template = '''
%INSTRUCTIONS:
<ADD_YOUR_INSTRUCTIONS_HERE>

%CONTEXT: {context}
%QUESTION: {input}
'''


)

# Combining Documents


In [20]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

# Final Retrieval Chain

In [21]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    

# Invoking the Retrieval Chain

In [22]:
response = retrieval_chain.invoke({"input": "Tell me name of monkeys and where do they live"})

In [23]:
response['answer']

"\nThe monkeys' names are Coco and Mango and they live in a tropical rainforest."

# Trying to list namespaces

In [31]:
pc_indx=pc.Index('uslaw')

In [32]:
pc_indx.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0},
                '0009-01.pdf': {'vector_count': 3},
                '0011-01.pdf': {'vector_count': 5},
                'monkey_story.txt': {'vector_count': 0},
                'xyz': {'vector_count': 0}},
 'total_vector_count': 8}

In [21]:
pcv= LangChainPinecone(index_name='uslaw',embedding=embeddings,pinecone_api_key=os.environ.get('PINECONE_API_KEY'))

In [24]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'testid-9g87gt2.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'testid',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 4096,
              'host': 'zain-9g87gt2.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'zain',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 4096,
              'host': 'immigration-9g87gt2.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'immigration',
              'spec': {'serverless': {'cloud': 'aws', 'regi